# Database Insert Notebook

In [56]:
import config
import get_zipcodes
import pandas as pd
import mysql.connector
from mysql.connector import errorcode
from bs4 import BeautifulSoup
import time
import re
import requests

## Extracting Dataset from CSV file and creating a Pandas DF

In [61]:
df = pd.read_csv('LoanStats_securev1_2019Q1.csv', header=1)
df.drop(df.tail(2).index,inplace=True)

/Users/kennymiyasato/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0,129,130,131,134,135,136,139,144,145,146) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [110]:
# final pick
df1 = df[['id','annual_inc','loan_amnt','purpose','application_type','grade','emp_length','tot_hi_cred_lim','home_ownership','addr_state','fico_range_low','dti','zip_code']]

# clean id to numeric
df1['id'] = pd.to_numeric(df1['id'])

# clean emp_length
df1 = df1.replace(to_replace=["< 1 year", "10+ years"], value=["0", "10"])
df1['emp_length'] = df1['emp_length'].str.replace('years','')
df1['emp_length'] = df1['emp_length'].str.replace('year','')
df1['emp_length'] = pd.to_numeric(df1['emp_length'])

# add more than 50k annual income column
def annual_inc_calculate(row):
    if row['annual_inc'] <= 50000:
        val = 0
    else:
        val = 1
    return val
df1['more_than_50k'] = df.apply(annual_inc_calculate, axis=1)

# limit dti
high_dti = df1[df1['dti'] > 100].index
df1.drop(high_dti, inplace=True)

# limit tot_hi_cred_lim
high_tot_hi_cred_lim = df1[df1['tot_hi_cred_lim'] > 1000000].index
df1.drop(high_tot_hi_cred_lim, inplace=True)

# create PK
df1['pk'] = df1['zip_code'].str.lower() + '_' + df1['addr_state'].str.lower()

#df re-ordered
df1 = df1[['id','pk','annual_inc','more_than_50k','purpose','application_type','grade','home_ownership','addr_state','zip_code','loan_amnt','emp_length','tot_hi_cred_lim','fico_range_low','dti']]

# dropnas
df1 = df1.dropna()

/Users/kennymiyasato/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [111]:
df1

,id,pk,annual_inc,more_than_50k,purpose,application_type,grade,home_ownership,addr_state,zip_code,loan_amnt,emp_length,tot_hi_cred_lim,fico_range_low,dti
0,149328059,152xx_pa,68000.0,1,debt_consolidation,Individual,A,MORTGAGE,PA,152xx,13500.0,8.0,224894.0,750.0,15.74
1,149420408,100xx_ny,110000.0,1,major_purchase,Individual,D,RENT,NY,100xx,20000.0,10.0,256497.0,675.0,23.44
2,149453166,787xx_tx,27840.0,0,other,Individual,B,RENT,TX,787xx,1600.0,1.0,46994.0,685.0,15.99
3,149491641,109xx_ny,28000.0,0,debt_consolidation,Individual,C,RENT,NY,109xx,6400.0,0.0,22700.0,700.0,8.62
4,149513578,704xx_la,130000.0,1,small_business,Joint App,A,MORTGAGE,LA,704xx,25000.0,0.0,348856.0,695.0,23.27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115668,145272394,194xx_pa,52500.0,1,debt_consolidation,Individual,C,MORTGAGE,PA,194xx,15000.0,10.0,193466.0,695.0,21.74
115671,143835299,060xx_ct,96000.0,1,debt_consolidation,Individual,A,MORTGAGE,CT,060xx,6000.0,0.0,33500.0,740.0,0.31
115672,145177297,925xx_ca,32000.0,0,debt_consolidation,Individual,C,MORTGAGE,CA,925xx,16000.0,10.0,34200.0,685.0,20.89
115673,145190854,850xx_az,72000.0,1,debt_consolidation,Individual,B,MORTGAGE,AZ,850xx,16000.0,5.0,251486.0,660.0,7.02


In [112]:
df1_to_tuples = list(df1.itertuples(index=False, name=None))

In [113]:
list_of_tuples_for_dataset = []
list_of_tuples_for_dataset.append(df1_to_tuples)

In [114]:
list_of_tuples_for_dataset

[[(149328059,
   '152xx_pa',
   68000.0,
   1,
   'debt_consolidation',
   'Individual',
   'A',
   'MORTGAGE',
   'PA',
   '152xx',
   13500.0,
   8.0,
   224894.0,
   750.0,
   15.74),
  (149420408,
   '100xx_ny',
   110000.0,
   1,
   'major_purchase',
   'Individual',
   'D',
   'RENT',
   'NY',
   '100xx',
   20000.0,
   10.0,
   256497.0,
   675.0,
   23.44),
  (149453166,
   '787xx_tx',
   27840.0,
   0,
   'other',
   'Individual',
   'B',
   'RENT',
   'TX',
   '787xx',
   1600.0,
   1.0,
   46994.0,
   685.0,
   15.99),
  (149491641,
   '109xx_ny',
   28000.0,
   0,
   'debt_consolidation',
   'Individual',
   'C',
   'RENT',
   'NY',
   '109xx',
   6400.0,
   0.0,
   22700.0,
   700.0,
   8.62),
  (149513578,
   '704xx_la',
   130000.0,
   1,
   'small_business',
   'Joint App',
   'A',
   'MORTGAGE',
   'LA',
   '704xx',
   25000.0,
   0.0,
   348856.0,
   695.0,
   23.27),
  (149514622,
   '077xx_nj',
   72500.0,
   1,
   'debt_consolidation',
   'Individual',
   'B',
   '

## Calling Census.gov's API for Median Income by County

### Defining Dictionaries to use

In [26]:
state = {'Alabama': 'AL','Alaska': 'AK','Arizona': 'AZ','Arkansas': 'AR',
         'California': 'CA','Colorado': 'CO','Connecticut': 'CT','Delaware': 'DE',
         'District of Columbia': 'DC','Florida': 'FL','Georgia': 'GA','Hawaii': 'HI',
         'Idaho': 'ID','Illinois': 'IL','Indiana': 'IN','Iowa': 'IA',
         'Kansas': 'KS','Kentucky': 'KY','Louisiana': 'LA','Maine': 'ME',
         'Maryland': 'MD','Massachusetts': 'MA','Michigan': 'MI','Minnesota': 'MN',
         'Mississippi': 'MS','Missouri': 'MO','Montana': 'MT','Nebraska': 'NE',
         'Nevada': 'NV','New Hampshire': 'NH','New Jersey': 'NJ','New Mexico': 'NM',
         'New York': 'NY','North Carolina': 'NC','North Dakota': 'ND','Ohio': 'OH',
         'Oklahoma': 'OK','Oregon': 'OR','Pennsylvania': 'PA','Puerto Rico': 'PR','Rhode Island': 'RI',
         'South Carolina': 'SC','South Dakota': 'SD','Tennessee': 'TN','Texas': 'TX',
         'Utah': 'UT','Vermont': 'VT','Virginia': 'VA','Washington': 'WA',
         'West Virginia': 'WV','Wisconsin': 'WI','Wyoming': 'WY'}

state_code = {  '01': 'AL', '02': 'AK', '04': 'AZ', '05': 'AR',
                '06': 'CA', '08': 'CO', '09': 'CT', '10': 'DE',
                '11': 'DC', '12': 'FL', '13': 'GA', '15': 'HI',
                '16': 'ID', '17': 'IL', '18': 'IN', '19': 'IA',
                '20': 'KS', '21': 'KY', '22': 'LA', '23': 'ME',
                '24': 'MD', '25': 'MA', '26': 'MI', '27': 'MN',
                '28': 'MS', '29': 'MO', '30': 'MT', '31': 'NE',
                '32': 'NV', '33': 'NH', '34': 'NJ', '35': 'NM',
                '36': 'NY', '37': 'NC', '38': 'ND', '39': 'OH',
                '40': 'OK', '41': 'OR', '42': 'PA', '44': 'RI',
                '45': 'SC', '46': 'SD', '47': 'TN', '48': 'TX',
                '49': 'UT', '50': 'VT', '51': 'VA', '53': 'WA',
                '54': 'WV', '55': 'WI', '56': 'WY', '72': 'PR'}

variable_name = "DP03_0062E"

### Creating function to call Census API

In [27]:
def call_census_api_for_counties(variable_name='DP03_0062E'):
    parameters = {
        "get": f"NAME,{variable_name}",
        "for": "county:*",
        "in" : "state:*",
        "key": config.census_api_key
    }

    response = requests.get(f"http://api.census.gov/data/2017/acs/acs5/profile", params=parameters)

    json_response = response.json()

    df = pd.DataFrame(json_response)
    df.columns = df.iloc[0]
    df = df.iloc[1:].reset_index()
    df = df.drop(['index'], axis=1)
    df['abbr'] = df['state'].map(state_code)
    
    for state_name in state:
        df['NAME'] = df['NAME'].str.replace(f', {state_name}', '')
        df['county_name'] = df['NAME'].str.replace('County', '')
        df['county_name'] = df['county_name'].str.replace(' Census Area', '')
        df['county_name'] = df['county_name'].str.replace(' City and Borough', '')
        df['county_name'] = df['county_name'].str.replace(' Borough', '')
        df['county_name'] = df['county_name'].str.replace(' Municipality', '')
        df['county_name'] = df['county_name'].str.replace(' Parish', '')
        df['county_name'] = df['county_name'].str.replace('St\.', 'Saint')
        df['county_name'] = df['county_name'].str.replace('-', ' ')
        df['county_name'] = df['county_name'].str.strip()

        
    df['PK']=df['county_name'].str.lower()+'_'+df['abbr'].str.lower()
    
    df = df[['NAME','abbr','state','DP03_0062E','county','county_name','PK']]
    
    list_of_tuples = []
    for index, row in df.iterrows():
        list_of_tuples.append((row['NAME'], row['abbr'], row['state'], row[f'{variable_name}'], row['county'], row['county_name'], row['PK']))
        
    return list_of_tuples, df

In [28]:
tuples, df_census_api_counties = call_census_api_for_counties()

In [29]:
list_of_tuples_for_census_api = []
list_of_tuples_for_census_api.append(tuples)

In [30]:
df_census_api_counties

,NAME,abbr,state,DP03_0062E,county,county_name,PK
0,Pickens County,AL,01,36220,107,Pickens,pickens_al
1,Sumter County,AL,01,21663,119,Sumter,sumter_al
2,Jefferson County,AL,01,49321,073,Jefferson,jefferson_al
3,Choctaw County,AL,01,32122,023,Choctaw,choctaw_al
4,Franklin County,AL,01,39501,059,Franklin,franklin_al
...,...,...,...,...,...,...,...
3215,Camuy Municipio,PR,72,17579,027,Camuy Municipio,camuy municipio_pr
3216,Ponce Municipio,PR,72,16561,113,Ponce Municipio,ponce municipio_pr
3217,Cataño Municipio,PR,72,18663,033,Cataño Municipio,cataño municipio_pr
3218,Florida Municipio,PR,72,16919,054,Florida Municipio,florida municipio_pr


## Webscraping getzips.com for Zipcode

### Create function to webscrape getzips.com

In [57]:
def get_zipcode(county, state):
    url = f"https://www.getzips.com/cgi-bin/ziplook.exe?What=3&County={county}&State={state}&Submit=Look+It+Up"
    page = requests.get(url)
    soup1 = BeautifulSoup(page.content, 'html.parser')

    table = soup1.find_all('table')[2]
    rows = table.find_all('tr')
    row_list = list()

    for tr in rows:
        td = tr.find_all('td')
        row = [i.text for i in td]
        row_list.append(row)

    columns = row_list[0]
    row_list.pop(0)
    try:
        first_3_zipcode = re.search("(^\d{3})+", row_list[0][0], flags=re.I)
        first_3_zipcode = first_3_zipcode[0]+"xx"
    except Exception as e:
        first_3_zipcode = '000xx'
        print(f"could not find: {county}, {state}")

    df_bs = pd.DataFrame(row_list,columns=columns)

    return (first_3_zipcode, county, state, county.lower()+'_'+state.lower())

In [60]:
dict_zipcodes_by_county = {}
for index in range(0, len(df_census_api_counties)):
    county_name = df_census_api_counties['county_name'][index].strip()
    state_name = df_census_api_counties['abbr'][index]
    dict_zipcodes_by_county[county_name] = get_zipcode(county_name, state_name)
    print(f'index: {index}   county_name: {county_name}   state: {state_name}')
    print(dict_zipcodes_by_county[county_name])
    time.sleep(.50)
dict_zipcodes_by_county

Note, since this webscrape is over 3,100 rows. It has been saved as a file called get_zipcodes.py

## Create Database

In [89]:
db_name = config.db_name

In [90]:
cnx = mysql.connector.connect(
    host=config.host,
    user=config.user,
    passwd=config.password,
    database=db_name,
    buffered=True
)

cursor = cnx.cursor()

In [91]:
def create_database(cursor, database):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)
        
#run create_database
try:
    cursor.execute("USE {}".format(db_name))
except mysql.connector.Error as err:
    print("Database {} does not exists.".format(db_name))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor, db_name)
        print("Database {} created successfully.".format(db_name))
        cnx.database = db_name
    else:
        print(err)
        exit(1)

## Create DB Tables

In [115]:
TABLES = {}
TABLES['census_median_income_data'] = (
    """CREATE TABLE census_median_income_data (
       full_county_name varchar(100) NOT NULL,
       state_abbrev      varchar(10) NOT NULL,
       state_code            int(10) NOT NULL,
       med_inc              int(100) NOT NULL,
       county_code           int(10) NOT NULL,
       county           varchar(100) NOT NULL,
       pk               varchar(100) NOT NULL,
       PRIMARY KEY (pk)
     ) ENGINE=InnoDB;""")

TABLES['zip_codes'] = (
    """CREATE TABLE zip_codes (
       zipcode           varchar(50) NOT NULL,
       county            varchar(50) NOT NULL,
       state_abbrev      varchar(10) NOT NULL,
       pk               varchar(100) NOT NULL,
       PRIMARY KEY (pk)
     ) ENGINE=InnoDB;""")

TABLES['dataset_lendingclub'] = (
    """CREATE TABLE dataset_lendingclub (
     id                         int(10) NOT NULL,
     pk                    varchar(100) NOT NULL,
     annual_inc                int(100) NOT NULL,
     more_than_50k              int(10) NOT NULL,
     purpose                varchar(50) NOT NULL,
     application_type       varchar(50) NOT NULL,
     grade                  varchar(10) NOT NULL,
     home_ownership         varchar(50) NOT NULL,
     addr_state             varchar(10) NOT NULL,
     zip_code               varchar(10) NOT NULL,
     loan_amnt                 int(100) NOT NULL,
     emp_length                 int(10) NOT NULL,
     tot_hi_cred_lim           int(100) NOT NULL,
     fico_range_low            int(100) NOT NULL,
     dti                        int(10) NOT NULL,
     PRIMARY KEY (pk)
     ) ENGINE=InnoDB;""")

In [116]:
for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")

Creating table census_median_income_data: already exists.
Creating table zip_codes: already exists.
Creating table dataset_lendingclub: OK


## Insert tuples into `dataset_lendingclub` db table

In [117]:
db_table = 'dataset_lendingclub'
for applicant in list_of_tuples_for_dataset:
    insert_statement = f"""INSERT INTO dataset_lendingclub 
                (
                    id,
                    annual_inc,
                    more_than_50k,
                    purpose,
                    application_type,
                    grade,
                    home_ownership,
                    addr_state,
                    zip_code,
                    loan_amnt,
                    emp_length,
                    tot_hi_cred_lim,
                    fico_range_low,
                    dti,
                    pk
                ) 
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""
    cursor.executemany(insert_statement, applicant)
    cnx.commit()
    print(f"Successfully added to database: '{db_name}' on table: '{db_table}' for county: {applicant[0]}")

IntegrityError: 1062 (23000): Duplicate entry '15.6' for key 'PRIMARY'

## Insert tuples into `census_median_income_data` db table

In [15]:
db_table = 'census_median_income_data'
for county in list_of_tuples_for_census_api:
    insert_statement = f"""INSERT INTO census_median_income_data 
                        (
                            full_county_name, 
                             state_abbrev, 
                             state_code, 
                             med_inc,
                             county_code, 
                             county, 
                             pk
                        ) 
                        VALUES (%s, %s, %s, %s, %s, %s, %s)"""
    cursor.executemany(insert_statement, county)
    cnx.commit()
    print(f"Successfully added to database: '{db_name}' on table: '{db_table}' for county: {county[0]}")

Successfully added to database: 'mod5_project' on table: 'census_median_income_data' for county: ('Pickens County', 'AL', '01', '36220', '107', 'Pickens', 'pickens_al')


## Insert tuples into `zip_code` db table

In [16]:
db_table = 'zip_codes'
for zipcode in get_zipcodes.zip_codes:
    insert_statement = f"""INSERT INTO zip_codes 
                        (
                            zipcode, 
                            county, 
                            state_abbrev, 
                            pk
                        ) 
                        VALUES (%s, %s, %s, %s)"""
    cursor.executemany(insert_statement, zipcode)
    cnx.commit()
    print(f"Successfully added to database: '{db_name}' on table: '{db_table}' for zipcode: {zipcode[0]}")

Successfully added to database: 'mod5_project' on table: 'zip_codes' for zipcode: ('354xx', 'Pickens', 'AL', 'pickens_al')
